## Imports

In [1]:
import numpy as np

from scipy import ndimage

from aoc_utilities import puzzle_input

<h2>--- Day 9: Smoke Basin ---</h2><p>These caves seem to be <a href="https://en.wikipedia.org/wiki/Lava_tube" target="_blank">lava tubes</a>. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly <span title="This was originally going to be a puzzle about watersheds, but we're already under water.">settles like rain</span>.</p>
<p>If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).</p>
<p>Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:</p>
<pre><code>2<em>1</em>9994321<em>0</em>
3987894921
98<em>5</em>6789892
8767896789
989996<em>5</em>678
</code></pre>
<p>Each number corresponds to the height of a particular location, where <code>9</code> is the highest and <code>0</code> is the lowest a location can be.</p>
<p>Your first goal is to find the <em>low points</em> - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)</p>
<p>In the above example, there are <em>four</em> low points, all highlighted: two are in the first row (a <code>1</code> and a <code>0</code>), one is in the third row (a <code>5</code>), and one is in the bottom row (also a <code>5</code>). All other locations on the heightmap have some lower adjacent location, and so are not low points.</p>
<p>The <em>risk level</em> of a low point is <em>1 plus its height</em>. In the above example, the risk levels of the low points are <code>2</code>, <code>1</code>, <code>6</code>, and <code>6</code>. The sum of the risk levels of all low points in the heightmap is therefore <code><em>15</em></code>.</p>
<p>Find all of the low points on your heightmap. <em>What is the sum of the risk levels of all low points on your heightmap?</em></p>


## Solution

Oh what the hell. Let's do Part 1 the fun way with a filter, even if we'll probably get burned by doing so in Part 2!

In [2]:
def keep_local_minima(array: np.ndarray):
    is_local_minimum = ndimage.filters.generic_filter(
        array,
        center_is_minimum,
        footprint=np.array([
            [0, 1, 0],
            [1, 1, 1],
            [0, 1, 0]
        ]),
        mode='constant',
        cval=np.infty,
        output=bool
    )
    return array[is_local_minimum]


def center_is_minimum(inputs):
    center_index = inputs.shape[0] // 2
    center, others = inputs[center_index], np.delete(inputs, center_index)
    return (center < others).all()

### Testing

In [3]:
example_heightmap_printout = '''
    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
'''

example_heightmap = np.array([
    [int(height) for height in line]
    for line in example_heightmap_printout.split()
])

assert (keep_local_minima(example_heightmap) + 1).sum() == 15

### Answer

In [4]:
heightmap = np.array([
    [int(height) for height in line]
    for line in puzzle_input.as_text(day=9).split()
])

print(
    'Sum of low point risk levels:',
    (keep_local_minima(heightmap) + 1).sum()
)

Sum of low point risk levels: 588


<h2 id="part2">--- Part Two ---</h2><p>Next, you need to find the largest basins so you know what areas are most important to avoid.</p>
<p>A <em>basin</em> is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height <code>9</code> do not count as being in any basin, and all other locations will always be part of exactly one basin.</p>
<p>The <em>size</em> of a basin is the number of locations within the basin, including the low point. The example above has four basins.</p>
<p>The top-left basin, size <code>3</code>:</p>
<pre><code><em>21</em>99943210
<em>3</em>987894921
9856789892
8767896789
9899965678
</code></pre>
<p>The top-right basin, size <code>9</code>:</p>
<pre><code>21999<em>43210</em>
398789<em>4</em>9<em>21</em>
985678989<em>2</em>
8767896789
9899965678
</code></pre>
<p>The middle basin, size <code>14</code>:</p>
<pre><code>2199943210
39<em>878</em>94921
9<em>85678</em>9892
<em>87678</em>96789
9<em>8</em>99965678
</code></pre>
<p>The bottom-right basin, size <code>9</code>:</p>
<pre><code>2199943210
3987894921
9856789<em>8</em>92
876789<em>678</em>9
98999<em>65678</em>
</code></pre>
<p>Find the three largest basins and multiply their sizes together. In the above example, this is <code>9 * 14 * 9 = <em>1134</em></code>.</p>
<p><em>What do you get if you multiply together the sizes of the three largest basins?</em></p>


## Solution
Well color me shocked!
Against all odds, Part 2 can also be solved using the ol' CV bag of tricks.

In [5]:
def basin_sizes(heightmap):
    return np.unique(
        ndimage.label(heightmap.max() - heightmap)[0],
        return_counts=True
    )[1][1:]


basin_sizes = basin_sizes(heightmap)
print(
    'Product of three largest basin sizes:',
    np.product(np.partition(basin_sizes, -3)[-3:])
)

Product of three largest basin sizes: 964712
